# Import Modules

In [1]:
import sys
import os
import pandas as pd
from datetime import timedelta
import math
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from sequential_testing import Sequential_AB_Helper
helper = Sequential_AB_Helper()

# Read data


In [2]:
#read the AdSmartABdata
data=pd.read_csv('../data/AdSmartABdata.csv')
data.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


# Transforming Data In to Bernoli Series


In [3]:
def transform_data(df):
    clean_data = df[df['yes'] == 1].append(df[df['no'] == 1]) 

    # Data Segmentation into exposed and control groups
    exposed_group = clean_data[clean_data['experiment'] == 'exposed']
    control_group = clean_data[clean_data['experiment'] == 'control']


    #Grouping Data using hours
    control_group['hour'] = control_group['hour'].astype('str')
    control_group['date'] = control_group['date'].astype('str')
    control_group['date+hour'] = pd.to_datetime(control_group['date'] + " " + control_group['hour'] + ":00:00")
    control_group['date+hour'] = control_group['date+hour'].map(lambda x:  pd.Timestamp(x, tz=None).strftime('%Y-%m-%d:%H'))

    exposed_group['hour'] = exposed_group['hour'].astype('str')
    exposed_group['date'] = exposed_group['date'].astype('str')
    exposed_group['date+hour'] = pd.to_datetime( exposed_group['date'] + " " + exposed_group['hour'] + ":00:00")
    exposed_group['date+hour'] = exposed_group['date+hour'].map( lambda x:  pd.Timestamp(x, tz=None).strftime('%Y-%m-%d:%H'))
   
    #Generating engagment and yes array based on date+hour column grouping
    control_counts = control_group.groupby('date+hour').agg({'yes': 'sum', 'no': 'count'})
   
    #Generate bernouli series
    control_group_bernouli = helper.bernouli_constructor(control_counts['no'],control_counts['yes'])

    exposed_counts = exposed_group.groupby('date+hour').agg({'yes': 'sum', 'no': 'count'})

    #Generate bernouli series
    exposed_group_bernouli = helper.bernouli_constructor(exposed_counts['no'],exposed_counts['yes'])

    data_stream = (control_group_bernouli, exposed_group_bernouli)

    return data_stream

In [4]:
data_stream = transform_data(data)
